In [1]:
import photon_mosaic_demo.widgets as pmw
from photon_mosaic_demo.extractors import read_scan_image_tiff
import matplotlib.pyplot as plt
%matplotlib widget


In [2]:
imaging = read_scan_image_tiff(file_path="C:/Users/ariellel/data/single-plane/neuron1_00001.tif",channel_name="Channel 1", plane_name='0')

In [ ]:
# Displays summary information about the movie
imaging

ScanImageTiffImaging (ROIExtractors) (ScanImageTiffImaging):
19.423000 Hz - 1 segments - 800 rows x 800 columns  samples - 13.7s - int16 dtype - 324.7MiB

In [ ]:
print(f"Number of frames {imaging.get_num_frames()}")
print(f"Image shape {imaging.image_shape}")
video_snippet = imaging.get_series(start_frame=0, end_frame=10) # demonstrating lazy access
print(f"Video snippet shape {video_snippet.shape}")


 Number of frames 266
 Image shape [800 800]
Video snippet shape (10, 800, 800)


In [ ]:
avg_image = imaging.get_average_image()
plt.figure()
plt.imshow(avg_image, vmax=20)

In [4]:
w = pmw.plot_imaging_series(imaging, backend="ipywidgets")

In [6]:
imaging_cached = imaging.save(folder="test", overwrite=True, chunk_duration="1s", n_jobs=1, verbose=True)

write_binary_imaging 
engine=process - n_jobs=1 - samples_per_chunk=19 - chunk_memory=11.60 MiB - total_memory=11.60 MiB - chunk_duration=0.98s (978.22 ms)


write_binary_imaging (no parallelization):   0%|          | 0/14 [00:00<?, ?it/s]

In [7]:
imaging.get_num_frames()

266

In [12]:
imaging.get_sampling_frequency()

19.423